In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a0456b21eb4743ce90c5441cdcf41162d0875031389625e942f5a891225e578f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, when, regexp_replace, regexp_extract, initcap, concat_ws, length, col, isnan, expr


import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
current_dir = os.getcwd()
print(current_dir)

/content


In [ ]:
spark = SparkSession.builder.getOrCreate()
print(spark)

In [ ]:
#Path to file csv
#/content/drive/MyDrive/transform_data_cellphones/raw/brands/brand_laptop.csv
#/content/drive/MyDrive/transform_data_cellphones/raw/products/laptop.csv
current_dir = "/content/drive/MyDrive/transform_data_cellphones"

In [ ]:
# function read csv
def read_csv(dir):
    data = spark.read \
        .format('csv') \
        .option('header', 'true') \
        .load(current_dir + dir)
    return data

In [ ]:
# function to csv
def to_csv(data, dir):
    data.to_csv(current_dir + dir, index=False)

In [ ]:
brand = read_csv('/raw/brands/brand_laptop.csv')

In [ ]:
brand.toPandas()

,brand_name,link,brand_id
0,mac,https://cellphones.com.vn/laptop/mac.html,1
1,asus,https://cellphones.com.vn/laptop/asus.html,2
2,msi,https://cellphones.com.vn/laptop/msi.html,3
3,lenovo,https://cellphones.com.vn/laptop/lenovo.html,4
4,hp,https://cellphones.com.vn/laptop/hp.html,5
5,acer,https://cellphones.com.vn/laptop/acer.html,6
6,surface,https://cellphones.com.vn/laptop/surface.html,7
7,lg,https://cellphones.com.vn/laptop/lg.html,8
8,huawei,https://cellphones.com.vn/laptop/huawei.html,9
9,dell,https://cellphones.com.vn/laptop/dell.html,10


In [ ]:
brand_for_laptop = brand.select('brand_name', 'link', 'brand_id')
brand.toPandas()

,brand_name,link,brand_id
0,mac,https://cellphones.com.vn/laptop/mac.html,1
1,asus,https://cellphones.com.vn/laptop/asus.html,2
2,msi,https://cellphones.com.vn/laptop/msi.html,3
3,lenovo,https://cellphones.com.vn/laptop/lenovo.html,4
4,hp,https://cellphones.com.vn/laptop/hp.html,5
5,acer,https://cellphones.com.vn/laptop/acer.html,6
6,surface,https://cellphones.com.vn/laptop/surface.html,7
7,lg,https://cellphones.com.vn/laptop/lg.html,8
8,huawei,https://cellphones.com.vn/laptop/huawei.html,9
9,dell,https://cellphones.com.vn/laptop/dell.html,10


In [ ]:
product = read_csv('/raw/products/laptop.csv')
product.toPandas().head()

,product_name,brand_id,product_link,current_price,original_price,percent_discount
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,1,https://cellphones.com.vn/macbook-pro-14-inch-...,49.990.000₫,0,giảm 0%
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,36.490.000₫,36.990.000₫,Giảm 1%
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,1,https://cellphones.com.vn/macbook-air-2020-m1....,18.690.000₫,22.990.000₫,Giảm 19%
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,1,https://cellphones.com.vn/macbook-air-m2-2022....,26.190.000₫,32.990.000₫,Giảm 21%
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,1,https://cellphones.com.vn/macbook-pro-m2-2022....,29.790.000₫,35.990.000₫,Giảm 17%


### **`current_price`**
* Regex: Extract numbers from the `current_price` column, removing non-numeric characters using `[^\d+]`.

In [ ]:
product = product.withColumn(
    'current_price', regexp_replace(col('current_price').cast('string'), r'[^\d+]', '').cast('integer')
).fillna(0)

product.toPandas()

,product_name,brand_id,product_link,current_price,original_price,percent_discount
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,1,https://cellphones.com.vn/macbook-pro-14-inch-...,49990000,0,giảm 0%
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,36490000,36.990.000₫,Giảm 1%
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,1,https://cellphones.com.vn/macbook-air-2020-m1....,18690000,22.990.000₫,Giảm 19%
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,1,https://cellphones.com.vn/macbook-air-m2-2022....,26190000,32.990.000₫,Giảm 21%
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,1,https://cellphones.com.vn/macbook-pro-m2-2022....,29790000,35.990.000₫,Giảm 17%
...,...,...,...,...,...,...
521,Laptop Vaio FE 14 VWNC51427-RG,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17.990.000₫,Giảm 22%
522,Laptop Vaio FE 14 VWNC51427-BL,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17.990.000₫,Giảm 22%
523,Laptop Vaio FE 15 VWNC51527-BK,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,giảm 0%
524,Laptop Vaio FE 15 VWNC51527-BL,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,giảm 0%


### **`original_price`**
* similar to `current_price`
* However, if the `original_price` is `NaN` or `original_price` = 0
  + if null: `original_price` will be set  = `current_price`
  + if not null: transformation will be applied.

In [ ]:
product = product.withColumn(
    'original_price',
    when((product.original_price.isNull()) | (product.original_price == "0") | isnan(product.original_price), product.current_price)
    .otherwise(regexp_replace(product.original_price, r'[^\d+]', '')).cast('integer')
)

product.toPandas()

,product_name,brand_id,product_link,current_price,original_price,percent_discount
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,1,https://cellphones.com.vn/macbook-pro-14-inch-...,49990000,49990000,giảm 0%
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,36490000,36990000,Giảm 1%
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,1,https://cellphones.com.vn/macbook-air-2020-m1....,18690000,22990000,Giảm 19%
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,1,https://cellphones.com.vn/macbook-air-m2-2022....,26190000,32990000,Giảm 21%
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,1,https://cellphones.com.vn/macbook-pro-m2-2022....,29790000,35990000,Giảm 17%
...,...,...,...,...,...,...
521,Laptop Vaio FE 14 VWNC51427-RG,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17990000,Giảm 22%
522,Laptop Vaio FE 14 VWNC51427-BL,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17990000,Giảm 22%
523,Laptop Vaio FE 15 VWNC51527-BK,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,giảm 0%
524,Laptop Vaio FE 15 VWNC51527-BL,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,giảm 0%


### **`percent_discount`**
* if `percent_discount` is null: will be set = 0
* else: will be transformed, by using `'\d+'`

In [ ]:
product = product.withColumn(
    'percent_discount', when(product.percent_discount.isNull(), 0) \
    .otherwise(regexp_extract(product.percent_discount, '\d+', 0)).cast('integer')
)

product.toPandas()

,product_name,brand_id,product_link,current_price,original_price,percent_discount
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,1,https://cellphones.com.vn/macbook-pro-14-inch-...,49990000,49990000,0
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,36490000,36990000,1
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,1,https://cellphones.com.vn/macbook-air-2020-m1....,18690000,22990000,19
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,1,https://cellphones.com.vn/macbook-air-m2-2022....,26190000,32990000,21
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,1,https://cellphones.com.vn/macbook-pro-m2-2022....,29790000,35990000,17
...,...,...,...,...,...,...
521,Laptop Vaio FE 14 VWNC51427-RG,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17990000,22
522,Laptop Vaio FE 14 VWNC51427-BL,12,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,13990000,17990000,22
523,Laptop Vaio FE 15 VWNC51527-BK,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,0
524,Laptop Vaio FE 15 VWNC51527-BL,12,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,0,0,0


In [ ]:
product.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- brand_id: string (nullable = true)
 |-- product_link: string (nullable = true)
 |-- current_price: integer (nullable = true)
 |-- original_price: integer (nullable = true)
 |-- percent_discount: integer (nullable = true)



In [ ]:
brand.toPandas().head()

,brand_name,link,brand_id
0,mac,https://cellphones.com.vn/laptop/mac.html,1
1,asus,https://cellphones.com.vn/laptop/asus.html,2
2,msi,https://cellphones.com.vn/laptop/msi.html,3
3,lenovo,https://cellphones.com.vn/laptop/lenovo.html,4
4,hp,https://cellphones.com.vn/laptop/hp.html,5


In [ ]:
product.toPandas().head()

,product_name,brand_id,product_link,current_price,original_price,percent_discount
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,1,https://cellphones.com.vn/macbook-pro-14-inch-...,49990000,49990000,0
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,36490000,36990000,1
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,1,https://cellphones.com.vn/macbook-air-2020-m1....,18690000,22990000,19
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,1,https://cellphones.com.vn/macbook-air-m2-2022....,26190000,32990000,21
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,1,https://cellphones.com.vn/macbook-pro-m2-2022....,29790000,35990000,17


In [ ]:
product = product.join(
    brand.select('brand_id', 'brand_name'),
    on='brand_id',
    how='inner'
)

In [ ]:
product.toPandas().head()

,brand_id,product_name,product_link,current_price,original_price,percent_discount,brand_name
0,1,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,https://cellphones.com.vn/macbook-pro-14-inch-...,49990000,49990000,0,mac
1,1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,https://cellphones.com.vn/imac-m3-2023-24-inch...,36490000,36990000,1,mac
2,1,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,https://cellphones.com.vn/macbook-air-2020-m1....,18690000,22990000,19,mac
3,1,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,https://cellphones.com.vn/macbook-air-m2-2022....,26190000,32990000,21,mac
4,1,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,https://cellphones.com.vn/macbook-pro-m2-2022....,29790000,35990000,17,mac


In [ ]:
product = product.select('product_name', 'current_price', 'original_price', 'percent_discount', 'product_link', 'brand_name')
product.toPandas()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,49990000,49990000,0,https://cellphones.com.vn/macbook-pro-14-inch-...,mac
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,36490000,36990000,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,mac
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,18690000,22990000,19,https://cellphones.com.vn/macbook-air-2020-m1....,mac
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,26190000,32990000,21,https://cellphones.com.vn/macbook-air-m2-2022....,mac
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,29790000,35990000,17,https://cellphones.com.vn/macbook-pro-m2-2022....,mac
...,...,...,...,...,...,...
521,Laptop Vaio FE 14 VWNC51427-RG,13990000,17990000,22,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,vaio
522,Laptop Vaio FE 14 VWNC51427-BL,13990000,17990000,22,https://cellphones.com.vn/laptop-vaio-fe-14-vw...,vaio
523,Laptop Vaio FE 15 VWNC51527-BK,0,0,0,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,vaio
524,Laptop Vaio FE 15 VWNC51527-BL,0,0,0,https://cellphones.com.vn/laptop-vaio-fe-15-vw...,vaio


In [ ]:
def product_tranform(dir_read, output_dir_product, brand_data):
    # Read CSV file
    product = read_csv(dir_read)

    # Current price
    product = product.withColumn(
    'current_price', regexp_replace(col('current_price').cast('string'), r'[^\d+]', '').cast('integer')
    ).fillna(0)

    # Original rice
    product = product.withColumn(
        'original_price',
        when((product.original_price.isNull()) | (product.original_price == "0") | isnan(product.original_price), product.current_price)
        .otherwise(regexp_replace(product.original_price, r'[^\d+]', '')).cast('integer')
    )

    # Percent discount
    product = product.withColumn(
        'percent_discount', when(product.percent_discount.isNull(), 0) \
        .otherwise(regexp_extract(product.percent_discount, '\d+', 0)).cast('integer')
    )

    # Merge brand data
    product = product.join(brand_data.select('brand_id', 'brand_name'), on='brand_id', how='inner')

    # Select columns
    product = product.select('product_name', 'current_price', 'original_price', 'percent_discount', 'product_link', 'brand_name')

    # to csv product
    to_csv(product.toPandas(), output_dir_product)

In [ ]:
brand_for_laptop = read_csv('/raw/brands/brand_laptop.csv')
brand_for_mobile = read_csv('/raw/brands/brand_mobile.csv')
brand_for_tablet = read_csv('/raw/brands/brand_tablet.csv')
brand_for_tv = read_csv('/raw/brands/brand_tv.csv')
brand_for_watch = read_csv('/raw/brands/brand_watch.csv')

In [ ]:
laptop = product_tranform('/raw/products/laptop.csv', '/transformed/products/laptop.csv', brand_for_laptop)
mobile = product_tranform('/raw/products/mobile.csv', '/transformed/products/mobile.csv', brand_for_mobile)
tablet = product_tranform('/raw/products/tablet.csv', '/transformed/products/tablet.csv', brand_for_tablet)
tv = product_tranform('/raw/products/tv.csv', '/transformed/products/tv.csv', brand_for_tv)
watch = product_tranform('/raw/products/watch.csv', '/transformed/products/watch.csv', brand_for_watch)

In [ ]:
df_laptop = read_csv('/transformed/products/laptop.csv')
df_laptop.toPandas().head()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,Macbook Pro 14 M3 Pro 18GB - 512GB | Chính hãn...,49990000,49990000,0,https://cellphones.com.vn/macbook-pro-14-inch-...,mac
1,iMac M3 24 inch 8GB 256GB 8GPU | Chính hãng Ap...,36490000,36990000,1,https://cellphones.com.vn/imac-m3-2023-24-inch...,mac
2,Apple MacBook Air M1 256GB 2020 I Chính hãng A...,18690000,22990000,19,https://cellphones.com.vn/macbook-air-2020-m1....,mac
3,Apple Macbook Air M2 2022 8GB 256GB I Chính hã...,26190000,32990000,21,https://cellphones.com.vn/macbook-air-m2-2022....,mac
4,Apple Macbook Pro 13 M2 2022 8GB 256GB I Chính...,29790000,35990000,17,https://cellphones.com.vn/macbook-pro-m2-2022....,mac


In [ ]:
df_mobile = read_csv('/transformed/products/mobile.csv')
df_mobile.toPandas().head()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,iPhone 15 Pro Max 256GB | Chính hãng VN/A,31990000,34990000,9,https://cellphones.com.vn/iphone-15-pro-max.html,apple
1,iPhone 15 128GB | Chính hãng VN/A,21590000,22990000,6,https://cellphones.com.vn/iphone-15.html,apple
2,iPhone 14 Pro Max 128GB | Chính hãng VN/A,26990000,29990000,10,https://cellphones.com.vn/iphone-14-pro-max.html,apple
3,iPhone 14 Pro 128GB | Chính hãng VN/A,25190000,27990000,10,https://cellphones.com.vn/iphone-14-pro.html,apple
4,iPhone 13 128GB | Chính hãng VN/A,15690000,18990000,17,https://cellphones.com.vn/iphone-13.html,apple


In [ ]:
df_tablet = read_csv('/transformed/products/tablet.csv')
df_tablet.toPandas().head()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,iPad Pro 11 inch 2022 M2 Wifi 128GB I Chính hã...,19990000,23990000,17,https://cellphones.com.vn/ipad-pro-2022-11-inc...,ipad
1,iPad 10.2 2021 WiFi 64GB | Chính hãng Apple Vi...,7090000,8990000,21,https://cellphones.com.vn/ipad-10-2-inch-2021....,ipad
2,iPad Air 5 10.9 inch (2022) WIFI 64GB I Chính ...,13790000,16990000,19,https://cellphones.com.vn/ipad-air-5.html,ipad
3,iPad mini 6 WiFi 64GB | Chính hãng Apple Việt Nam,11590000,12990000,11,https://cellphones.com.vn/ipad-mini-6.html,ipad
4,iPad Pro 11 inch 2022 M2 Wifi 256GB I Chính hã...,23790000,26990000,12,https://cellphones.com.vn/ipad-pro-2022-11-inc...,ipad


In [ ]:
df_tv = read_csv('/transformed/products/tv.csv')
df_tv.toPandas().head()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,Smart Tivi di động LG Stanby Me 27inch,17790000,22990000,23,https://cellphones.com.vn/smart-tivi-lg-standb...,lg
1,Smart Tivi LG UHD 4K 55 inch 55UQ7050,8490000,13900000,39,https://cellphones.com.vn/smart-tivi-lg-uhd-55...,lg
2,Smart Tivi LG UHD 4K 50 inch 50UQ7050,7990000,11900000,33,https://cellphones.com.vn/smart-tivi-lg-uhd-50...,lg
3,Smart Tivi LG UHD 4K 43 inch 43UQ7050,7190000,10900000,34,https://cellphones.com.vn/smart-tivi-lg-uhd-43...,lg
4,Smart Tivi NanoCell LG 4K 55 inch 55NANO76SQA,11390000,22900000,50,https://cellphones.com.vn/smart-tivi-nanocell-...,lg


In [ ]:
df_watch = read_csv('/transformed/products/watch.csv')
df_watch.toPandas().head()

,product_name,current_price,original_price,percent_discount,product_link,brand_name
0,Apple Watch Series 9 45mm (GPS) viền nhôm dây ...,10590000,11290000,6,https://cellphones.com.vn/apple-watch-series-9...,apple-watch
1,Apple Watch SE 2023 2 44mm (GPS) viền nhôm | C...,6790000,7190000,6,https://cellphones.com.vn/apple-watch-se-2023-...,apple-watch
2,Apple Watch SE 2 2023 40mm (GPS) viền nhôm | C...,5690000,6390000,11,https://cellphones.com.vn/apple-watch-se-2023-...,apple-watch
3,Apple Watch Series 9 41mm (GPS) viền nhôm dây ...,9590000,10490000,9,https://cellphones.com.vn/apple-watch-series-9...,apple-watch
4,Apple Watch Series 8 41mm GPS viền nhôm | Chín...,9190000,11990000,23,https://cellphones.com.vn/apple-watch-series-8...,apple-watch
